In [1]:
import pandas as pd, numpy as np

In [18]:
w_metadata_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/lsff_input_coverage_data_with_metadata.csv'
df = pd.read_csv(w_metadata_path)

In [19]:
df.head()

,location_id,location_name,sub_population,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,u5_applicable,wra_applicable,estimation_status,source_citation,source_link,CI_source,data_choice_notes
0,179,Ethiopia,NaN,oil,percent of population eating fortified vehicle,vitamin a,1.0,0.0,5.0,True,True,NaN,NaN,NaN,NaN,NaN
1,179,Ethiopia,NaN,wheat flour,percent of population eating fortified vehicle,folic acid,1.0,0.0,5.0,True,True,NaN,NaN,NaN,NaN,NaN
2,179,Ethiopia,under-5,wheat flour,percent of population eating industrially prod...,na,27.0,17.0,37.0,True,False,NaN,NaN,NaN,NaN,NaN
3,179,Ethiopia,women of reproductive age,wheat flour,percent of population eating industrially prod...,na,20.0,10.0,30.0,False,True,NaN,NaN,NaN,NaN,NaN
4,179,Ethiopia,under-5,oil,percent of population eating industrially prod...,na,70.0,60.0,80.0,True,True,NaN,NaN,NaN,NaN,NaN


In [49]:
df[(df.estimation_status=='regression')]

,location_id,location_name,sub_population,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,u5_applicable,wra_applicable,estimation_status,source_citation,source_link,CI_source,data_choice_notes,data_richness
70,196,South Africa,NaN,oil,percent of population eating fortified vehicle,vitamin a,34.013318,0.000000,35.198719,True,True,regression,NaN,NaN,modeling,NaN,0
71,196,South Africa,NaN,oil,percent of population eating industrially prod...,na,81.716956,0.000000,84.564880,True,True,regression,NaN,NaN,modeling,NaN,0
72,196,South Africa,NaN,oil,percent of population eating vehicle,na,99.014877,0.000000,100.000000,True,True,regression,NaN,NaN,modeling,NaN,0
89,201,Burkina Faso,NaN,maize flour,percent of population eating fortified vehicle,vitamin a,0.000000,0.000000,0.000000,True,True,regression,NaN,NaN,modeling,NaN,0
99,201,Burkina Faso,NaN,wheat flour,percent of population eating fortified vehicle,vitamin a,12.388563,0.000000,26.275167,True,True,regression,NaN,NaN,modeling,NaN,0
100,201,Burkina Faso,NaN,wheat flour,percent of population eating fortified vehicle,zinc,12.388563,0.000000,26.275167,True,True,regression,NaN,NaN,modeling,NaN,0
109,180,Kenya,NaN,oil,percent of population eating fortified vehicle,vitamin a,0.000000,0.000000,0.000000,True,True,regression,na,na,modeling,NaN,0
116,15,Myanmar,NaN,oil,percent of population eating vehicle,na,76.774138,0.000000,100.000000,True,True,regression,NaN,NaN,modeling,NaN,0
123,15,Myanmar,NaN,wheat flour,percent of population eating fortified vehicle,vitamin a,0.000000,0.000000,0.000000,True,True,regression,NaN,NaN,modeling,NaN,0
126,164,Nepal,NaN,oil,percent of population eating industrially prod...,na,74.747650,0.000000,100.000000,True,True,regression,NaN,NaN,modeling,NaN,0


In [20]:
df['data_richness'] = [0 if i=='regression' else 1 for i in df.estimation_status]

In [21]:
output = df.groupby(['location_name','vehicle','value_description','nutrient']).sum()[['data_richness']]

In [22]:
output

data_richness
location_name vehicle     value_description                                  nutrient                 
Bangladesh    oil         percent of population eating fortified vehicle     vitamin a               1
                          percent of population eating industrially produ... na                      1
                          percent of population eating vehicle               na                      1
              wheat flour percent of population eating fortified vehicle     folic acid              1
                                                                             iron                    1
...                                                                                                ...
Viet Nam      wheat flour percent of population eating fortified vehicle     iron                    1
                                                                             vitamin a               1
                                                                             zinc                    1
                          percent of population eating industrially produ... na                      1
                          percent of population eating vehicle               na                      2

[209 rows x 1 columns]

In [23]:
output.sort_values('data_richness')

data_richness
location_name vehicle     value_description                                  nutrient                 
Cameroon      wheat flour percent of population eating fortified vehicle     vitamin a               0
              maize flour percent of population eating fortified vehicle     folic acid              0
                                                                             iron                    0
                                                                             vitamin a               0
                                                                             zinc                    0
...                                                                                                ...
Bangladesh    wheat flour percent of population eating vehicle               na                      2
Ethiopia      wheat flour percent of population eating industrially produ... na                      2
Cameroon      wheat flour percent of population eating vehicle               na                      2
Burkina Faso  wheat flour percent of population eating vehicle               na                      2
Viet Nam      wheat flour percent of population eating vehicle               na                      2

[209 rows x 1 columns]

In [24]:
output = output.reset_index()

In [25]:
C = output[(output.value_description=="percent of population eating fortified vehicle")].drop(columns='value_description').rename(columns={'data_richness':'C'})
B = output[(output.value_description=="percent of population eating industrially produced vehicle")].drop(columns=['value_description','nutrient']).rename(columns={'data_richness':'B'})
A = output[(output.value_description=="percent of population eating vehicle")].drop(columns=['value_description','nutrient']).rename(columns={'data_richness':'A'})

In [26]:
C.nutrient.unique()

array(['vitamin a', 'folic acid', 'iron', 'zinc'], dtype=object)

In [34]:
data_rich = A.merge(B, on = ['location_name','vehicle'], how = 'left').merge(C, on = ['location_name','vehicle'], how = 'outer').fillna(0)

In [35]:
data_rich

,location_name,vehicle,A,B,nutrient,C
0,Bangladesh,oil,1.0,1.0,vitamin a,1
1,Bangladesh,wheat flour,2.0,1.0,folic acid,1
2,Bangladesh,wheat flour,2.0,1.0,iron,1
3,Bangladesh,wheat flour,2.0,1.0,vitamin a,1
4,Bangladesh,wheat flour,2.0,1.0,zinc,1
...,...,...,...,...,...,...
120,Nigeria,oil,0.0,0.0,vitamin a,1
121,Nigeria,wheat flour,0.0,0.0,folic acid,1
122,Nigeria,wheat flour,0.0,0.0,iron,1
123,Nigeria,wheat flour,0.0,0.0,vitamin a,1


In [36]:
for col in ['A','B','C']:
    data_rich[col] = ((2 ** data_rich[col]) + 1) % 2

In [37]:
data_rich

,location_name,vehicle,A,B,nutrient,C
0,Bangladesh,oil,1.0,1.0,vitamin a,1
1,Bangladesh,wheat flour,1.0,1.0,folic acid,1
2,Bangladesh,wheat flour,1.0,1.0,iron,1
3,Bangladesh,wheat flour,1.0,1.0,vitamin a,1
4,Bangladesh,wheat flour,1.0,1.0,zinc,1
...,...,...,...,...,...,...
120,Nigeria,oil,0.0,0.0,vitamin a,1
121,Nigeria,wheat flour,0.0,0.0,folic acid,1
122,Nigeria,wheat flour,0.0,0.0,iron,1
123,Nigeria,wheat flour,0.0,0.0,vitamin a,1


In [40]:
data_rich['score'] = data_rich[['A','B','C']].sum(axis=1)

In [43]:
data_rich = data_rich[['location_name','vehicle','nutrient','score']].sort_values(['score'])

In [44]:
data_rich

,location_name,vehicle,nutrient,score
17,Cameroon,maize flour,zinc,0.0
41,Indonesia,oil,vitamin a,0.0
26,Côte d'Ivoire,maize flour,zinc,0.0
25,Côte d'Ivoire,maize flour,vitamin a,0.0
24,Côte d'Ivoire,maize flour,iron,0.0
...,...,...,...,...
53,Kenya,wheat flour,vitamin a,3.0
52,Kenya,wheat flour,iron,3.0
51,Kenya,wheat flour,folic acid,3.0
61,Mozambique,wheat flour,iron,3.0


In [45]:
data_rich[(data_rich.score==0)]

,location_name,vehicle,nutrient,score
17,Cameroon,maize flour,zinc,0.0
41,Indonesia,oil,vitamin a,0.0
26,Côte d'Ivoire,maize flour,zinc,0.0
25,Côte d'Ivoire,maize flour,vitamin a,0.0
24,Côte d'Ivoire,maize flour,iron,0.0
23,Côte d'Ivoire,maize flour,folic acid,0.0
44,Indonesia,wheat flour,vitamin a,0.0
16,Cameroon,maize flour,vitamin a,0.0
15,Cameroon,maize flour,iron,0.0
14,Cameroon,maize flour,folic acid,0.0


In [46]:
data_rich[(data_rich.score==1)]

,location_name,vehicle,nutrient,score
113,Ethiopia,wheat flour,iron,1.0
122,Nigeria,wheat flour,iron,1.0
42,Indonesia,wheat flour,folic acid,1.0
111,Ethiopia,oil,vitamin a,1.0
43,Indonesia,wheat flour,iron,1.0
112,Ethiopia,wheat flour,folic acid,1.0
121,Nigeria,wheat flour,folic acid,1.0
120,Nigeria,oil,vitamin a,1.0
119,India,wheat flour,zinc,1.0
114,Ethiopia,wheat flour,zinc,1.0


In [47]:
data_rich[(data_rich.score==2)]

,location_name,vehicle,nutrient,score
69,Nepal,oil,vitamin a,2.0
67,Myanmar,wheat flour,vitamin a,2.0
36,Democratic Republic of the Congo,oil,vitamin a,2.0
50,Kenya,oil,vitamin a,2.0
64,Myanmar,oil,vitamin a,2.0
39,Democratic Republic of the Congo,wheat flour,vitamin a,2.0
106,Viet Nam,oil,vitamin a,2.0
28,Côte d'Ivoire,wheat flour,folic acid,2.0
59,Mozambique,oil,vitamin a,2.0
7,Burkina Faso,maize flour,vitamin a,2.0


In [48]:
data_rich[(data_rich.score==3)]

,location_name,vehicle,nutrient,score
90,Uganda,maize flour,vitamin a,3.0
89,Uganda,maize flour,iron,3.0
88,Uganda,maize flour,folic acid,3.0
82,South Africa,maize flour,zinc,3.0
91,Uganda,maize flour,zinc,3.0
...,...,...,...,...
53,Kenya,wheat flour,vitamin a,3.0
52,Kenya,wheat flour,iron,3.0
51,Kenya,wheat flour,folic acid,3.0
61,Mozambique,wheat flour,iron,3.0


In [56]:
save_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/data_richness_score.csv'
data_rich.to_csv(save_path, index = False)

In [ ]:
/ihme/homes/beatrixh/repos/scratch/data_richness_score